# Tutorial about logging

Logging as supplied by the python standard library can be used.

We make use of the standard logging levels DEBUG, INFO, WARNING, ERROR, CRITICAL.

In [ ]:
import logging
import sys
from pathlib import Path

import numpy as np
import pandas as pd

import locan as lc

In [ ]:
lc.show_versions(dependencies=False, verbose=False)

## Activate logging

In any script or notebook logging has to be enabled e.g. for streaming to stdout.

For changing the configuration logging has to be reloaded or the kernel be restarted.

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

A top-level logger has to be instantiated to process any log messages from the library.

In [ ]:
logger = logging.getLogger()
logger

Further log messages can be added:

In [ ]:
logger.info("test")

### Handling locan.logger

To change the filter level of locan log records, use an instance of the locan logger identified by its module name.

In [ ]:
locan_logger = logging.getLogger('locan')
locan_logger.setLevel(logging.INFO)
locan_logger

## Logging in locan

Many functions provide warnings if some unusual behavior occurs:

In [ ]:
locdata = lc.LocData.from_coordinates([(0, 0), (1, 2), (2, 1), (5, 5)])
locdata.region = lc.Rectangle((0, 0), 2, 2, 0)

Changing the level of the locan logger to `logging.WARNING` or higher, will switch off most locan log records.

In [ ]:
locan_logger.setLevel(logging.ERROR)

locdata = lc.LocData.from_coordinates([(0, 0), (1, 2), (2, 1), (5, 5)])
locdata.region = lc.Rectangle((0, 0), 2, 2, 0)

In [ ]:
locan_logger.setLevel(logging.INFO)

locdata = lc.LocData.from_coordinates([(0, 0), (1, 2), (2, 1), (5, 5)])
locdata.region = lc.Rectangle((0, 0), 2, 2, 0)

Levels can be set for selected loggers by specifying the module or function name:

In [ ]:
logger_locan_data = logging.getLogger('locan.data')
logger_locan_data.setLevel(logging.ERROR)

locdata = lc.LocData.from_coordinates([(0, 0), (1, 2), (2, 1), (5, 5)])
locdata.region = lc.Rectangle((0, 0), 2, 2, 0)

In [ ]:
logger_locan_data.setLevel(logging.INFO)

locdata = lc.LocData.from_coordinates([(0, 0), (1, 2), (2, 1), (5, 5)])
locdata.region = lc.Rectangle((0, 0), 2, 2, 0)

### Logging in a pipeline

In [ ]:
def computation(self, file):
    logger.info(f'computation started for file: {file}')
    return self

In [ ]:
pipes = [lc.Pipeline(computation=computation, file=file).compute() for file in range(3)]

Another example how to use logging in analysis pipelines is given by the `computation_test` function.

In [ ]:
pipes = [lc.Pipeline(computation=lc.analysis.pipeline.computation_test, locdata=file).compute() for file in range(3)]

In [ ]:
print(pipes[0].computation_as_string())

### Logging in multiprocessing with ray

To enable logging in multiprocessing using ray you need to include a default configuration in the computation function: `logging.basicConfig(level=logging.INFO)`.

In [ ]:
if False:
    def computation(self, file):
        logging.basicConfig(level=logging.INFO)
        logger.info(f'computation started for file: {file}')
        return self

In [ ]:
if False:
    import ray

    ray.init()
    # ray.init(num_cpus = 4)

In [ ]:
%%time
if False:
    @ray.remote
    def worker(file):
        pipe = lc.Pipeline(computation=computation, file=file).compute()
        return pipe

    futures = [worker.remote(file) for file in range(3)]
    pipes = ray.get(futures)
    len(pipes)

## Logging in locan - third party libraries

Some third-party libraries provide their own logging system. Typically the individual loggers can be imported and modified.

In [ ]:
import trackpy as tr

In [ ]:
tr.logger

alternatively

In [ ]:
trackpy_logger = logging.getLogger('trackpy')
trackpy_logger

Depending on the library various methods can be used to change the logging level. All of the following can be used.

`trackpy_logger.setLevel(logging.WARN)`

`tr.logger.setLevel(logging.WARN)`

`tr.quiet()`

`tr.ignore_logging()`  # this switches off the trackpy logging system and forwards all logs up the logger hirarchy.

In [ ]:
dat = lc.simulate_tracks(n_walks=1, n_steps=100, ranges=((0,1000),(0,1000)),
                      diffusion_constant=1, seed=1)

dat.print_meta()

In [ ]:
locdata_new, track_series = lc.track(dat, search_range=5)

In [ ]:
trackpy_logger.setLevel(logging.WARN)
locdata_new, track_series = lc.track(dat, search_range=5)